# Curso de Optimización (DEMAT)
## Tarea 8

| Descripción:                         | Fechas               |
|--------------------------------------|----------------------|
| Fecha de publicación del documento:  | **Abril 11, 2022**   |
| Fecha límite de entrega de la tarea: | **Mayo   1, 2022**   |


### Indicaciones

- Envie el notebook que contenga los códigos y las pruebas realizadas de cada ejercicio.
- Si se requiren algunos scripts adicionales para poder reproducir las pruebas,
  agreguelos en un ZIP junto con el notebook.
- Genere un PDF del notebook y envielo por separado.

---

## Ejercicio 1 (3 puntos)

Sea $x=(x_1, x_2, ..., x_n)$ la variable independiente.

Programar las siguientes funciones y sus gradientes:

- Función cuadrática 

$$ f(\mathbf{x}) = 0.5\mathbf{x}^\top \mathbf{A}\mathbf{x} - \mathbf{b}^\top\mathbf{x}. $$

Si $\mathbf{I}$ es la matriz identidad y $\mathbf{1}$ es la matriz llena de 1's,
ambas de tamaño $n$, entonces

$$ \mathbf{A} = n\mathbf{I} + \mathbf{1} = 
\left[\begin{array}{llll} n      & 0      & \cdots & 0 \\
                       0      & n      & \cdots & 0 \\ 
                       \vdots & \vdots & \ddots & \vdots \\
                       0      & 0      & \cdots & n \end{array}\right]
+ \left[\begin{array}{llll} 1    & 1      & \cdots & 1 \\
                       1      & 1      & \cdots & 1 \\ 
                       \vdots & \vdots & \ddots & \vdots \\
                       1      & 1      & \cdots & 1 \end{array}\right],  \qquad
\mathbf{b} = \left[\begin{array}{l} 1 \\ 1 \\ \vdots \\ 1 \end{array}\right] $$


- Función generalizada de Rosenbrock

$$  f(x) = \sum_{i=1}^{n-1} 100(x_{i+1} - x_i^2)^2 + (1 - x_{i} )^2  $$

$$ x_0 = (-1.2, 1, -1.2, 1, ..., -1.2, 1) $$


En la implementación de cada función y de su gradiente, se recibe como argumento la variable $x$
y definimos $n$ como la longitud del arreglo $x$, y con esos datos aplicamos la 
definición correspondiente.

Estas funciones van a ser usadas para probar los algoritmos de optimización.
El  punto $x_0$ que aparece en la definición de cada función es el punto inicial
que se sugiere para el algoritmo de optimización.


### Solución:

In [68]:
import numpy as np

# Implementación de la función cuadrática y su gradiente

def cuad(x):
    N = x.shape[1]
    
    b = np.ones((1,N))
    A = np.identity(N)*N + np.ones((N,N))
    return 0.5 * x @ A @ x.T - b @ x.T

def g_cuad(x):
    N = x.shape[1]
    
    b = np.ones((N,))
    A = np.identity(N)*N + np.ones((N,N))
    
    return x @ A - b

In [30]:
# Implementación de la función tridiagonal generalizada y su gradiente
import numpy as np

N = 5

b = np.ones((1,N))
x = np.array([[1,2,3,4,5]])
print(x)

print(x[:,1:], x[:,:-1])

[[1 2 3 4 5]]
[[2 3 4 5]] [[1 2 3 4]]


In [34]:
#  Implementación de la función generalizada de Rosenbrock y su gradiente

def rosenbrock(x):
    x1 = x[:,:-1]
    x2 = x[:,1:]
    return np.sum(100 * (x2 - x1**2)**2 + (1-x1)**2)
    
def g_rosenbrock(x):
    N = x.shape[1]
    
    x1 = x[:,:-1]
    x2 = x[:,1:]
    
    d1 = -400 * (x2 - x1**2) * x1 - 2 * (1 - x1)
    d2 = 200 * (x2 - x1**2)
    
    d = np.zeros((1,N))
    d[:,0] = d1[:,0]
    d[:,1:-1] = d1[:,1:] + d2[:,:-1]
    d[:,-1] = d2[:,-1]
    
    return d


```


```
---


## Ejercicio 2 (3.5 puntos)

Programar el método de gradiente conjugado no lineal de Fletcher-Reeves:

---

La implementación recibe como argumentos a la función objetivo $f$, su gradiente $\nabla f$,
un punto inicial $x_0$, el máximo número de iteraciones $N$ y una tolerancia $\tau>0$.

1. Calcular  $\nabla f_0 = \nabla f(x_0)$, $p_0 = -\nabla f_0$ y hacer $res=0$.
2. Para $k=0,1,..., N$:

- Si $\|\nabla f_k\|< \tau$, hacer $res=1$ y terminar el ciclo
- Usando backtracking calcular el tamaño de paso  $\alpha_k$
- Calcular $x_{k+1} = x_k + \alpha_k p_k$
- Calcular $\nabla f_{k+1} = \nabla f(x_{k+1})$
- Calcular 

$$ \beta_{k+1} = \frac{\nabla f_{k+1}^\top \nabla f_{k+1}}{\nabla f_{k}^\top\nabla f_{k}}  $$ 

- Calcular 

$$ p_{k+1} = -\nabla f_{k+1} + \beta_{k+1} p_k $$

3. Devolver $x_k, \nabla f_k, k, res$

```


```
---

1. Escriba la función que implemente el algoritmo anterior.
2. Pruebe el algoritmo usando para cada una de las funciones del 
   Ejercicio 1, tomando el punto $x_0$ que se indica.
3. Fije $N=50000$, $\tau = \epsilon_m^{1/3}$.
4. Para cada función del Ejercicio 1 cree el punto $x_0$ correspondiente
   usado $n=2, 10, 20$ y ejecute el algoritmo.
   Imprima
   
- n,
- f(x0),
- las primeras y últimas 4 entradas del punto $x_k$ que devuelve el algoritmo,
- f(xk),
- la norma del vector $\nabla f_k$, 
- el  número $k$ de iteraciones realizadas,
- la variable $res$ para saber si el algoritmo puedo converger.
  

### Solución:

In [71]:
# En esta celda puede poner el código de las funciones
# o poner la instrucción para importarlas de un archivo .py

def backtracking(f, xk, pk, p, beta):
    alpha = 1
    fk = f(xk)
    while True:
        if f(xk + alpha * pk) <= fk - beta*alpha*(pk @ pk.T):
            return alpha
        alpha = p * alpha

def Fletcher_Reeves(f, g, x0, maxN, tol):
    xk = x0
    res = 0
    
    gnext = g(xk)
    pk = -gnext
    
    for k in range(maxN):
        gk = gnext
        if np.linalg.norm(gk) < tol:
            res = 1
            break

        alpha = backtracking(f, xk, pk, 0.8, 0.0001)
        xk = xk + alpha * pk
        
        gnext = g(xk)
        betak = (gnext @ gnext.T) / (gk @ gk.T)
        pk = -gnext + betak * pk
        
        if np.abs(gnext @ gk.T) > 0.2 * np.linalg.norm(gnext)**2:
            # Reinicio
            pk = - gnext
    
    return xk, gk, k, res

In [75]:
# Pruebas realizadas 

EPS_M = np.finfo(float).eps
MAX_N = 50000
TOL = EPS_M ** (1/3)

def short(x):
    print(np.concatenate((x[0,:3], x[0,-min(3, len(x)-3):]if len(x) > 3 else [])))


def test_Fletcher_Reeves(N, f, g, maxN, tol, mess):
    x0 = np.ones((1,N))
    x0[:,::2] = -1.2
    
    print(f"Test Fletcher-Reeves | {mess}")
    print(f'N = {N}')
    print(x0)
    print(f'f(x0) = {f(x0)}')
    
    xk, gk, k, res = Fletcher_Reeves(f, g, x0, maxN, tol)
    
    print('xk = ', end='')
    short(xk)
    print(f'||gk|| = {np.linalg.norm(gk)}')
    print(f'f(xk) = {f(xk)}')
    print(f'k = {k}')
    print(f'res = {res}')
    print()
    
nn = (2, 10, 20)
for n in nn:
    test_Fletcher_Reeves(n, cuad, g_cuad, MAX_N, TOL, 'cuadrática')
print()
for n in nn:
    test_Fletcher_Reeves(n, rosenbrock, g_rosenbrock, MAX_N, TOL, 'Rosenbrock')

Test Fletcher-Reeves | cuadrática
N = 2
[[-1.2  1. ]]
f(x0) = [[2.66]]
xk = [0.24999919 0.24999919]
||gk|| = 4.601667816800304e-06
f(xk) = [[-0.25]]
k = 32
res = 1

Test Fletcher-Reeves | cuadrática
N = 10
[[-1.2  1.  -1.2  1.  -1.2  1.  -1.2  1.  -1.2  1. ]]
f(x0) = [[62.5]]
xk = [0.04999993 0.04999993 0.04999993]
||gk|| = 4.668968258165778e-06
f(xk) = [[-0.25]]
k = 50
res = 1

Test Fletcher-Reeves | cuadrática
N = 20
[[-1.2  1.  -1.2  1.  -1.2  1.  -1.2  1.  -1.2  1.  -1.2  1.  -1.2  1.
  -1.2  1.  -1.2  1.  -1.2  1. ]]
f(x0) = [[248.]]
xk = [0.02500003 0.02500003 0.02500003]
||gk|| = 4.800317012941582e-06
f(xk) = [[-0.25]]
k = 63
res = 1


Test Fletcher-Reeves | Rosenbrock
N = 2
[[-1.2  1. ]]
f(x0) = 24.199999999999996
xk = [1.00000294 1.00000588]
||gk|| = 6.040269183391008e-06
f(xk) = 8.656373449210375e-12
k = 16816
res = 1

Test Fletcher-Reeves | Rosenbrock
N = 10
[[-1.2  1.  -1.2  1.  -1.2  1.  -1.2  1.  -1.2  1. ]]
f(x0) = 2057.0
xk = [0.99999999 0.99999998 0.99999997]
||gk|| = 


```


```
---

## Ejercicio 3 (3.5 puntos)

Programar el método de gradiente conjugado no lineal de usando la fórmula de
Hestenes-Stiefel:

En este caso el algoritmo es igual al del Ejercicio 2, con excepción del cálculo de $\beta_{k+1}$. Primero se calcula el vector $\mathbf{y}_k$ y luego $\beta_{k+1}$:

$$ \mathbf{y}_k =  \nabla f_{k+1}-\nabla f_{k} $$
$$ \beta_{k+1} =   \frac{\nabla f_{k+1}^\top\mathbf{y}_k }{\nabla p_{k}^\top\mathbf{y}_k}  $$

1. Repita el Ejercicio 2 usando la fórmula de Hestenes-Stiefel.
2. ¿Cuál de los métodos es mejor para encontrar los óptimos de las funciones de prueba?

### Solución:

In [77]:
# En esta celda puede poner el código de las funciones
# o poner la instrucción para importarlas de un archivo .py

def Hestenes_Stiefel(f, g, x0, maxN, tol):
    xk = x0
    res = 0
    
    gnext = g(xk)
    pk = -gnext
    
    for k in range(maxN):
        gk = gnext
        if np.linalg.norm(gk) < tol:
            res = 1
            break

        alpha = backtracking(f, xk, pk, 0.9, 0.0001)
        xk = xk + alpha * pk
        
        gnext = g(xk)
        
        yk = gnext - gk
        betak = (gnext @ yk.T) / (pk @ yk.T)
        pk = -gnext + betak * pk
        
        if np.abs(gnext @ gk.T) > 0.2 * np.linalg.norm(gnext)**2:
            # Reinicio
            pk = - gnext
    
    return xk, gk, k, res

In [78]:
# Pruebas realizadas 

EPS_M = np.finfo(float).eps
MAX_N = 50000
TOL = EPS_M ** (1/3)

def test_Hestenes_Stiefel(n, f, g, maxN, tol, mess):
    x0 = np.ones((1,N))
    x0[:,::2] = -1.2
    
    print(f"Test Hestenes_Stiefel | {mess}")
    print(f'n = {n}')
    print(f'f(x0) = {f(x0)}')
    
    xk, gk, k, res = Hestenes_Stiefel(f, g, x0, maxN, tol)
    
    print('xk = ', end='')
    short(xk)
    print(f'||gk|| = {np.linalg.norm(gk)}')
    print(f'f(xk) = {f(xk)}')
    print(f'k = {k}')
    print(f'res = {res}')
    print()
    
nn = (2, 10, 20)
for n in nn:
    test_Hestenes_Stiefel(n, cuad, g_cuad, MAX_N, TOL, 'cuadrática')
print()
for n in nn:
    test_Hestenes_Stiefel(n, rosenbrock, g_rosenbrock, MAX_N, TOL, 'Rosenbrock')

Test Hestenes_Stiefel | cuadrática
n = 2
f(x0) = [[18.68]]
xk = [0.09999974 0.09999974 0.09999974]
||gk|| = 5.873056436555319e-06
f(xk) = [[-0.25]]
k = 96
res = 1

Test Hestenes_Stiefel | cuadrática
n = 10
f(x0) = [[18.68]]
xk = [0.09999974 0.09999974 0.09999974]
||gk|| = 5.873056436555319e-06
f(xk) = [[-0.25]]
k = 96
res = 1

Test Hestenes_Stiefel | cuadrática
n = 20
f(x0) = [[18.68]]
xk = [0.09999974 0.09999974 0.09999974]
||gk|| = 5.873056436555319e-06
f(xk) = [[-0.25]]
k = 96
res = 1


Test Hestenes_Stiefel | Rosenbrock
n = 2
f(x0) = 1016.4000000000001
xk = [0.99999982 0.99999963 0.99999926]
||gk|| = 6.025060792347923e-06
f(xk) = 2.9892103437503606e-12
k = 19971
res = 1

Test Hestenes_Stiefel | Rosenbrock
n = 10
f(x0) = 1016.4000000000001
xk = [0.99999982 0.99999963 0.99999926]
||gk|| = 6.025060792347923e-06
f(xk) = 2.9892103437503606e-12
k = 19971
res = 1

Test Hestenes_Stiefel | Rosenbrock
n = 20
f(x0) = 1016.4000000000001
xk = [0.99999982 0.99999963 0.99999926]
||gk|| = 6.025060


```


```
---

### Conclusión

Notemos que con ambos métodos se obtuvo un resultado convergente en todas las pruebas, más aún, llegaban al mismo mínimo. La única parte donde se desempeñaron distinto fue en el número de iteraciones. Notemos que el método *Fletcher-Reeves* siempre tuvo menos iteraciones que *Hestenes-Stiefel*. Por lo tanto en estas funciones resulto mejor usar el método *Fletcher-Reeves*.